# 📊📄📈 PyWin32 Office 자동화 실습 노트북 (C:\ 루트 저장 + PowerPoint)

이 노트북은 Windows 환경에서 **C 드라이브 루트**에 파일을 저장하며, PyWin32를 사용해
- Excel 자동화 3가지
- PowerPoint 성적표 자동 생성
- Word 자동화 3가지
- Outlook Gemini API 자동 답장

예제를 포함합니다.


In [1]:
# 필요한 패키지 설치 (최초 1회)
# !pip install pywin32 google-genai pandas openpyxl
print("패키지 설치 확인")

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 29.2 MB/s eta 0:00:00

   ----------------------------------------  0/11 [websockets]
   ------------------ ---------------------  5/11 [annotated-types]
   --------------------- ------------------  6/11 [pydantic]
   --------------------- ------------------  6/11 [pydantic]
   ------------------------- --------------  7/11 [httpcore]
   ----------------------------- ----------  8/11 [anyio]
   -------------------------------- -------  9/11 [httpx]
   ------------------------------------ --- 10/11 [google-genai]
   ---------------------------------------- 11/11 [google-genai]

패키지 설치 확인


In [12]:
# 📈 Excel 예제 1: 새 통합 문서 생성 → C:\학생점수.xlsx 저장
import win32com.client as win32

excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Add()
ws = wb.Worksheets('Sheet1')

data = [['이름','국어','영어','수학'],['철수',90,85,92],['영희',88,79,95],['민수',75,80,78]]
for r, row in enumerate(data, start=1):
    for c, val in enumerate(row, start=1):
        ws.Cells(r,c).Value = val

path = r'C:\학생점수.xlsx'
wb.SaveAs(path)
print("저장:", path)
wb.Close(False); excel.Quit()

저장: C:\학생점수.xlsx


In [22]:
# 📈 Excel 예제 2: C:\학생점수.xlsx 열기 → B2 +5 점 → 저장
import win32com.client as win32, os

path = r'C:\학생점수.xlsx'
if not os.path.exists(path):
    raise FileNotFoundError(path + " 없음")

excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Open(path)
ws = wb.Worksheets('Sheet1')

orig = ws.Range("B2").Value
ws.Range("B2").Value = orig + 5
wb.Save()
print("B2 업데이트:", ws.Range("B2").Value)

wb.Close(False); excel.Quit()

B2 업데이트: 140.0


In [27]:
import win32com.client as win32
import os

# --- 1) 원본 파일 경로 & 출력 파일 경로 설정 ---
file1 = r'sales_transactions.xlsx'
file2 = r'support_tickets.xlsx'
output_path = r'combined.xlsx'

# --- 2) Excel 실행 ---
excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = False  # 작업 중 엑셀 창을 보려면 True

# --- 3) 새 통합 문서 생성 & Placeholder 시트 추가 ---
wb = excel.Workbooks.Add()
placeholder = wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))
placeholder.Name = 'Placeholder'

# --- 4) 기본 생성 시트(Placeholder 제외) 삭제 ---
#    이제 기본 시트가 하나 이상 남아 있어도 오류가 나지 않습니다.
for sh in list(wb.Worksheets):
    if sh.Name != 'Placeholder':
        sh.Delete()

# --- 5) 첫 번째 파일의 모든 시트 복사 ---
wb1 = excel.Workbooks.Open(file1)
for sh in wb1.Worksheets:
    sh.Copy(After=wb.Sheets(wb.Sheets.Count))
wb1.Close(False)

# --- 6) 두 번째 파일의 모든 시트 복사 ---
wb2 = excel.Workbooks.Open(file2)
for sh in wb2.Worksheets:
    sh.Copy(After=wb.Sheets(wb.Sheets.Count))
wb2.Close(False)

# --- 7) Placeholder 시트 삭제 ---
wb.Sheets('Placeholder').Delete()

# --- 8) 통합 문서 저장 & 종료 ---
os.makedirs(os.path.dirname(output_path), exist_ok=True)
wb.SaveAs(output_path)
wb.Close(False)
excel.Quit()

print(f'✅ 통합 파일 저장 완료: {output_path}')


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: ''

In [23]:
# 📈 Excel 예제 3: C:\학생점수.xlsx 차트 생성 → C:\chart.png
import win32com.client as win32, os

path = r'C:\학생점수.xlsx'
if not os.path.exists(path):
    raise FileNotFoundError(path + " 없음")

excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Open(path); ws = wb.Worksheets('Sheet1')

chart = wb.Charts.Add()
chart.ChartType = 51  # 막대형
chart.SetSourceData(Source=ws.Range("B2:D4"))
chart.HasTitle = True
chart.ChartTitle.Text = "학생별 점수"
img = r'C:\\chart.png'
chart.Export(img)
print("차트 저장:", img)

wb.Close(False); excel.Quit()

차트 저장: C:\\chart.png


## 📊 PowerPoint 자동화 예제 – 성적표 슬라이드 생성

Excel 예제에서 만든 **C:\학생점수.xlsx**를 읽어
PowerPoint 성적표 슬라이드를 자동 생성합니다.

In [28]:
# PowerPoint 성적표 자동 생성
import win32com.client as win32
import pandas as pd

excel_path = r'C:\학생점수.xlsx'
ppt_path   = r'C:\GradeReport.pptx'

df = pd.read_excel(excel_path)
ppt = win32.gencache.EnsureDispatch('PowerPoint.Application')
ppt.Visible = True
pres = ppt.Presentations.Add()
slide = pres.Slides.Add(1, 1)  # 제목 슬라이드
slide.Shapes.Title.TextFrame.TextRange.Text = "학생 성적표"

table = slide.Shapes.AddTable(df.shape[0]+1, df.shape[1], 50,150,600,300).Table
# 헤더
for col, name in enumerate(df.columns, start=1):
    table.Cell(1,col).Shape.TextFrame.TextRange.Text = name
# 데이터
for r in range(df.shape[0]):
    for c, val in enumerate(df.iloc[r], start=1):
        table.Cell(r+2,c).Shape.TextFrame.TextRange.Text = str(val)

pres.SaveAs(ppt_path)
print("PowerPoint 저장:", ppt_path)

PowerPoint 저장: C:\GradeReport.pptx


In [29]:
# 📄 Word 예제 1: C:\AutoReport.docx 생성
import win32com.client as win32
word = win32.gencache.EnsureDispatch('Word.Application'); word.Visible=True
doc = word.Documents.Add()
sel = word.Selection
sel.Style = doc.Styles("제목 1"); sel.TypeText("자동 보고서\n\n")
sel.Style = doc.Styles("본문"); sel.TypeText("PyWin32 자동 생성 문서")
path = r'C:\\AutoReport.docx'; doc.SaveAs(path)
print("저장:",path)
doc.Close(False); word.Quit()

저장: C:\\AutoReport.docx


In [ ]:
# 📄 Word 예제 2: 차트 삽입
import win32com.client as win32, os
doc_path=r'C:\\AutoReport.docx'; img=r'C:\\chart.png'
word=win32.gencache.EnsureDispatch('Word.Application'); word.Visible=True
doc=word.Documents.Open(doc_path); sel=word.Selection; sel.EndKey(6)
sel.InlineShapes.AddPicture(img); sel.ParagraphFormat.Alignment=1
doc.Save(); print("이미지 삽입 완료")
doc.Close(False); word.Quit()

In [30]:
# 📄 Word 예제 3: '보고서'→'리포트', C:\AutoReport.pdf 저장
import win32com.client as win32
word=win32.gencache.EnsureDispatch('Word.Application'); word.Visible=True
doc=word.Documents.Open(r'C:\\AutoReport.docx')
find=word.Selection.Find; find.Text="보고서"; find.Replacement.Text="리포트"; find.Execute(Replace=2)
path=r'C:\\AutoReport.pdf'; doc.SaveAs(path, FileFormat=17)
print("PDF 저장:",path)
doc.Close(False); word.Quit()

PDF 저장: C:\\AutoReport.pdf


In [31]:
# ✉️ Outlook 자동 답장 (Gemini API)
import win32com.client as win32
from google import genai; from google.genai import types
client=genai.Client(api_key='GEMINI_API_KEY')
model="gemini-2.5-pro-preview-03-25"
outlook=win32.gencache.EnsureDispatch("Outlook.Application")
sel=outlook.ActiveExplorer().Selection
if sel.Count==0: raise RuntimeError("메일 선택 필요")
mail=sel.Item(1); orig=mail.Body
prompt=f"공손하게 답장 (한국어):\n\n{orig}"
contents=[types.Content(role="user",parts=[types.Part.from_text(prompt)])]
cfg=types.GenerateContentConfig(response_mime_type="text/plain")
reply_text=""
for ch in client.models.generate_content_stream(model=model,contents=contents,config=cfg):
    reply_text+=ch.text
rep=mail.Reply(); rep.Body=reply_text+"\n\n(자동생성)"
rep.Display()
print("답장 생성")

AttributeError: 'NoneType' object has no attribute 'Selection'

In [35]:
# extract_pdf_text.py

import sys
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path: str):
    """
    주어진 PDF 파일 경로를 열어, 페이지별 텍스트를 추출해 터미널(또는 셀 출력)에 표시합니다.
    """
    try:
        reader = PdfReader(pdf_path)
    except FileNotFoundError:
        print(f"[Error] 파일을 찾을 수 없습니다: {pdf_path}", file=sys.stderr)
        return
    except Exception as e:
        print(f"[Error] PDF 열기 실패: {e}", file=sys.stderr)
        return

    print(f"\n✅ PDF 열기 성공: {pdf_path} (총 {len(reader.pages)} 페이지)\n")
    for i, page in enumerate(reader.pages, start=1):
        text = page.extract_text()
        print(f"--- 페이지 {i} ---")
        if text:
            print(text.strip())
        else:
            print("[이 페이지에 텍스트가 없습니다]")
        print()

def get_pdf_path_from_args():
    """
    sys.argv 에서 -- 로 시작하지 않는 첫 번째 값을 PDF 경로로 간주.
    """
    raw = sys.argv[1:]
    # Jupyter 등의 내부 옵션(--f=...) 은 제거
    candidates = [arg for arg in raw if not arg.startswith('-')]
    return candidates[0] if candidates else None

if __name__ == "__main__":
    # 1) CLI 인자 우선
    pdf_file = get_pdf_path_from_args()
    # 2) CLI 인자가 없으면 대화형 입력
    if not pdf_file:
        pdf_file = input("▶ 추출할 PDF 파일 경로를 입력하세요: ").strip()
    extract_text_from_pdf(pdf_file)



✅ PDF 열기 성공: C:\\AutoReport.pdf (총 1 페이지)

--- 페이지 1 ---
자동 리포트  
 
PyWin32 자동 생성 문서

